In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-20b",groq_api_key=groq_api_key)
model

c:\Users\battu\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001EFF6F46A70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EFF6F46980>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Harish")])


AIMessage(content='Hello, Harish! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'User says: "Hi, My name is Harish". They are introducing themselves. We should respond politely, greet them, maybe ask how we can help.'}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 78, 'total_tokens': 134, 'completion_time': 0.056668979, 'completion_tokens_details': {'reasoning_tokens': 33}, 'prompt_time': 0.004256931, 'prompt_tokens_details': None, 'queue_time': 0.053572168, 'total_time': 0.06092591}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--8a9efc00-a9da-4946-a00c-99cf2911cf62-0', usage_metadata={'input_tokens': 78, 'output_tokens': 56, 'total_tokens': 134, 'output_token_details': {'reasoning': 33}})

In [6]:
from langchain_core.messages import AIMessage
model.invoke([HumanMessage(content="Hi, My name is Harish"), AIMessage(content="Hello Harish, How can I help you?"),HumanMessage(content="What's my name ?")])

AIMessage(content='Your name is Harish.', additional_kwargs={'reasoning_content': 'The user says "Hi, My name is Harish" then asks "What\'s my name?" We should respond with "Your name is Harish." This is straightforward.'}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 102, 'total_tokens': 152, 'completion_time': 0.051866468, 'completion_tokens_details': {'reasoning_tokens': 35}, 'prompt_time': 0.00558965, 'prompt_tokens_details': None, 'queue_time': 0.594662519, 'total_time': 0.057456118}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e45d7342-6d4f-4555-bdf3-3eeb478e7437-0', usage_metadata={'input_tokens': 102, 'output_tokens': 50, 'total_tokens': 152, 'output_token_details': {'reasoning': 35}})

In [10]:
### Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [12]:
response=with_message_history.invoke([HumanMessage(content="Hi, My name is Harish")],config=config)

In [15]:
response.content

'Hi Harish! 👋 How can I help you today? If you have any questions or need assistance with something, just let me know!'

In [19]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Answer the all Questions"),
    MessagesPlaceholder(variable_name="messages")
])
chain=prompt|model


In [20]:
chain.invoke({
    "messages":[
        HumanMessage(content="Hi, My name is Harish")
    ]
})

AIMessage(content='Hello Harish! 👋 How can I assist you today?', additional_kwargs={'reasoning_content': 'User says: "Hi, My name is Harish". Likely wants introduction. We need to respond politely, greet, maybe ask how to help.'}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 92, 'total_tokens': 146, 'completion_time': 0.055779911, 'completion_tokens_details': {'reasoning_tokens': 32}, 'prompt_time': 0.005106805, 'prompt_tokens_details': None, 'queue_time': 0.053269274, 'total_time': 0.060886716}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--762e6f12-d1ba-44e4-9ee3-7879c997a258-0', usage_metadata={'input_tokens': 92, 'output_tokens': 54, 'total_tokens': 146, 'output_token_details': {'reasoning': 32}})

In [21]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [23]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([HumanMessage(content="Hi, My name is Harish")], config=config)
response.content



'Nice to meet you, Harish! How can I assist you today?'

In [24]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Answer the all Questions {language}"),
    MessagesPlaceholder(variable_name="messages")
])
chain=prompt|model


In [25]:
response=chain.invoke({
    "messages":[
        HumanMessage(content="Hi, My name is Harish")
    ],
    "language":"telugu"
})

In [26]:
response.content

'హలో హరిష్! మీకు ఏమైనా ప్రశ్నలు ఉంటే, దయచేసి అడగండి. నేను తెలుగులో మీకు సహాయం చేయడానికి సిద్ధంగా ఉన్నాను.'

In [27]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history,input_messages_key="messages")

In [29]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, My name is Harish")], 'language':"telugu"}, config=config)


In [30]:
response.content

'హాయ్ హరిష! మీరు ఎలా ఉన్నారు? ఏమైనా ప్రశ్నలు ఉంటే తెలుగులో అడగండి, నేను సహాయం చేస్తాను.'

In [31]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="What's my name ?")], 'language':"telugu"}, config=config)


In [32]:
response.content

'మీ పేరు హరిష్.'

In [40]:
### Managing the Chat History
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"

)
messages=[
    SystemMessage(content="You are a helpful AI assistant. Answer the all Questions"),
    HumanMessage(content="Hi, My name is Harish"),
    AIMessage(content="Hello Harish, How can I help you?"),
    HumanMessage(content="What's my name ?")
]
response=trimmer.invoke(messages)
response

[SystemMessage(content='You are a helpful AI assistant. Answer the all Questions', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, My name is Harish', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Harish, How can I help you?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's my name ?", additional_kwargs={}, response_metadata={})]

In [43]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(
        history=itemgetter("messages")|trimmer
    )
    |prompt|model
)
response = chain.invoke({
    "messages":messages + [HumanMessage(content="What's my name ?")],
    "language":"telugu"
})
response.content


'మీ పేరు హరిష.'